In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 30)

## Defining an aggregation
#### All grouping operation have those three components: the grouping columns, aggregating columns, and aggregating functions.
#### Here we'll look at the simplest possible aggregation  a single grouping column, a single aggregating column, and a single aggregating function. We will find the average arrival delay for each airline.

In [2]:
flights = pd.read_csv('master/data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [3]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'})
# This returns a single column DF

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580
F9,13.630651
HA,4.972973
MQ,6.860591
NK,18.436070


In [152]:
# Exploration of the index and columns, to complete

ind_exp = flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'})

# Comme tout DF, il est composé d'un index qui à un nom

ind_exp.index

Index(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA', 'US',
       'VX', 'WN'],
      dtype='object', name='AIRLINE')

In [5]:
# d'un index horizontal (columns)

ind_exp.columns

Index(['ARR_DELAY'], dtype='object')

In [6]:
# Et de valeurs

ind_exp.values

array([[ 5.54266055],
       [-0.83333333],
       [ 8.69259259],
       [ 0.33969067],
       [ 7.0345796 ],
       [13.63065134],
       [ 4.97297297],
       [ 6.86059143],
       [18.43606999],
       [ 7.59346304],
       [ 7.76575521],
       [ 1.68110483],
       [ 5.34888438],
       [ 6.39735259]])

In [7]:
# Alternatively
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean')
# This syntax returns a Series

AIRLINE
AA     5.542661
AS    -0.833333
B6     8.692593
DL     0.339691
EV     7.034580
F9    13.630651
HA     4.972973
MQ     6.860591
NK    18.436070
OO     7.593463
UA     7.765755
US     1.681105
VX     5.348884
WN     6.397353
Name: ARR_DELAY, dtype: float64

In [8]:
ind_exp2 = flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean')
ind_exp2.index

Index(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA', 'US',
       'VX', 'WN'],
      dtype='object', name='AIRLINE')

In [9]:
ind_exp2.values

array([ 5.54266055, -0.83333333,  8.69259259,  0.33969067,  7.0345796 ,
       13.63065134,  4.97297297,  6.86059143, 18.43606999,  7.59346304,
        7.76575521,  1.68110483,  5.34888438,  6.39735259])

In [10]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [11]:
# When you are only applying a single aggregating function, we can also use directly methods of the groupby object.
# Not all aggregation functions have a method equivalent but many basic ones do.
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [12]:
# If you do not use an aggregating function with agg, pandas raises an exception.
# flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.sqrt)

## Grouping and aggregating with multiple columns and functions
#### We'll answering the following queries:
    - Finding the number of cancelled flights for every airline per weekday
    
    - Finding the number and percentage of cancelled and diverted flights for every airline per weekday
    
    - For each origin and destination, finding the total number of flights, the number and percentage of cancelled flights,
      and the average and variance of the airtime

In [13]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [14]:
# Finding the number of cancelled flights for every airline per weekday

flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'] \
       .agg('sum').head(21)

# As there is only one aggregating column AND only one aggregating function, a Series is returned

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
         6          21
         7          29
AS       1           0
         2           0
         3           0
         4           0
         5           0
         6           0
         7           0
B6       1           0
         2           1
         3           0
         4           0
         5           0
         6           0
         7           0
Name: CANCELLED, dtype: int64

In [15]:
# Notons que notre Series a à présent un index composé d'un MultiIndex à 2 levels

gb_insp = flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'] \
                 .agg('sum').head(21)

gb_insp.index

MultiIndex(levels=[['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA', 'US', 'VX', 'WN'], [1, 2, 3, 4, 5, 6, 7]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2], [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]],
           names=['AIRLINE', 'WEEKDAY'])

In [16]:
# Finding the number and percentage of cancelled and diverted flights for every airline per weekday

flights.groupby(['AIRLINE', 'WEEKDAY']) \
        ['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head(21)

# As there are more than one aggregating column AND/OR more than one aggregating function, a DF is returned

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
        6              21  0.018667        9  0.008000
        7              29  0.021837        1  0.000753
AS      1               0  0.000000        0  0.000000
        2               0  0.000000        0  0.000000
        3               0  0.000000        0  0.000000
        4               0  0.000000        0  0.000000
        5               0  0.000000        0  0.000000
        6               0  0.000000        0  0.000000
        7               0  0.000000        0  0.000000
B6      1               0  0.000000        0  0.000000
        2               1  0.012658        0  0.000000
        3               0  0.000000        0  0.000000
        4               0  0.000000        0  0.000000
        5               0  0.000000        0  0.000000
        6               0  0.000000        1  0.016667
        7               0  0.000000        1  0.011364

In [17]:
# Notons que notre DF a un index et des columns tous 2 composés de MultiIndex à 2 levels:

gpDF_insp = flights.groupby(['AIRLINE', 'WEEKDAY']) \
        ['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head(21)

gpDF_insp.columns

MultiIndex(levels=[['CANCELLED', 'DIVERTED'], ['sum', 'mean']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [18]:
# For each origin and destination, finding the total number of flights, the number
# and percentage of cancelled flights, and the average and variance of the airtime

group_cols = ['ORG_AIR', 'DEST_AIR']
agg_dict = {'CANCELLED':['sum', 'mean', 'size'],
            'AIR_TIME':['mean', 'var']}
flights.groupby(group_cols).agg(agg_dict).tail(7)

# Notice that the size aggregating function returns the total number of rows per group, which is
# different from the count aggregating function, which returns the number of non-missing values per group.

CANCELLED                   AIR_TIME            
                       sum      mean size        mean         var
ORG_AIR DEST_AIR                                                 
SFO     SMF              2  0.033898   59   21.052632    8.729323
        SMX              1  0.071429   14   40.846154    3.641026
        SNA              4  0.032787  122   64.059322   11.338331
        STL              0  0.000000   20  198.900000  101.042105
        SUN              0  0.000000   10   78.000000   25.777778
        TUS              0  0.000000   20  100.200000   35.221053
        XNA              0  0.000000    2  173.500000    0.500000

In [19]:
# Test perso
flights.query(" ORG_AIR =='SFO' and DEST_AIR =='STL' ")

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
1402,1,9,5,OO,SFO,STL,1610,147.0,204.0,1735,2158,142.0,0,0
1941,1,13,2,OO,SFO,STL,1610,151.0,222.0,1735,2158,169.0,0,0
6333,2,9,1,OO,SFO,STL,1610,41.0,183.0,1735,2158,17.0,0,0
13360,3,22,7,WN,SFO,STL,945,-1.0,194.0,1735,1535,-23.0,0,0
16042,4,6,1,WN,SFO,STL,945,-4.0,192.0,1735,1535,-29.0,0,0
19412,4,25,6,WN,SFO,STL,800,-3.0,194.0,1735,1350,-22.0,0,0
25745,5,30,6,WN,SFO,STL,800,-2.0,205.0,1735,1350,-3.0,0,0
32510,7,6,1,WN,SFO,STL,1035,40.0,220.0,1735,1625,46.0,0,0
33376,7,10,5,UA,SFO,STL,1100,4.0,199.0,1735,1703,-16.0,0,0
34262,7,15,3,UA,SFO,STL,1100,0.0,203.0,1735,1703,-9.0,0,0


### TM

In [20]:
# If you do not specify the aggregating columns, then the aggregating method will be applied to all the non-grouping columns:

flights.groupby(['ORG_AIR', 'DEST_AIR']).sum().head()

MONTH  DAY  WEEKDAY  SCHED_DEP  DEP_DELAY  AIR_TIME   DIST  \
ORG_AIR DEST_AIR                                                               
ATL     ABE         167  519      132      46567      298.0    2988.0  21452   
        ABQ          98  189       62      25426      113.0    2728.0  20304   
        ABY         136  345       87      31253      427.0     543.0   2755   
        ACY          40   72       11       8972       43.0     548.0   4074   
        AEX         275  627      160      59674      440.0    3149.0  20000   

                  SCHED_ARR  ARR_DELAY  DIVERTED  CANCELLED  
ORG_AIR DEST_AIR                                             
ATL     ABE           52916      185.0         0          0  
        ABQ           27960        4.0         0          0  
        ABY           32758      391.0         0          0  
        ACY           10096       64.0         0          0  
        AEX           62361      301.0         0          0

## Removing the MultiIndex after grouping
#### We'll manipulate a DataFrame with a MultiIndex for the rows and columns so that the index becomes a single level and the column names are descriptive.

In [21]:
flights = pd.read_csv('master/data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [22]:
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY'])\
                      .agg({'DIST':['sum', 'mean'],
                           'ARR_DELAY':['min', 'max']}) \
                      .astype(int)
airline_info.head(11)

# As we see Pandas displays MultiIndex levels differently than single-level columns.
# Except for the innermost levels, repeated index values do not get displayed on the screen.

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732
        6        1265340  1124       -50  858
        7        1461906  1100       -49  626
AS      1         121478  1029       -28  145
        2         114681  1081       -38   86
        3         120150  1044       -45  106
        4         122273  1111       -35  344

In [23]:
airline_info.columns

MultiIndex(levels=[['DIST', 'ARR_DELAY'], ['max', 'mean', 'min', 'sum']],
           labels=[[0, 0, 1, 1], [3, 1, 2, 0]])

In [24]:
level0 = airline_info.columns.get_level_values(0)
level0

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [25]:
level1 = airline_info.columns.get_level_values(1)
level1

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [26]:
# Indexes support vectorized operations

airline_info.columns = level0 + '_' + level1
airline_info.head()

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732

In [27]:
# Here we could have concatenated the levels together like we did in step 2, but it makes more sense like this:

airline_info.reset_index().head(3)

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473


In [ ]:
airline_info.reset_index()

#### By default, at the end of a groupby operation, pandas puts all of the grouping columns in the index. The as_index parameter in the groupby method can be set to False to avoid this behavior. Let's try by finding the average distance traveled per flight from each airline:

In [28]:
flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean') \
                                                    .round(0)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


## Customizing an aggregation function
#### In this recipe, we use the college dataset to calculate the mean and standard deviation of the colleges undergraduate student population per state. We then use this information to find the maximum number of standard deviations from the mean that any single population value is per state. We will then determine what is the maximum standard deviation that a college population have in each group.

In [29]:
college = pd.read_csv('master/data/college.csv')
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [30]:
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']) \
                                 .round(0).head(11)

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0
CA,3518.0,6709.0
CO,2325.0,4670.0
CT,1874.0,2871.0
DC,2645.0,3225.0


In [31]:
col_ca_cond = college['STABBR'] == 'CA'
col_ca = college[col_ca_cond]
col_ca_s = col_ca['UGDS']
col_ca_s.mean()

3518.3083969465647

In [32]:
c_s = ((col_ca_s - col_ca_s.mean()) / col_ca_s.std())
c_s.abs().head()

192    0.949039
193    0.449918
194         NaN
195    0.522363
196    0.486737
Name: UGDS, dtype: float64

In [33]:
c_s.abs().max()

6.145228956365477

In [34]:
# For each state group, we will take each population,
# substract the mean of the group and divide this result by the std of the group.
# We don't care wether this number is negative or positive because we just try to determine what is the maximum
# std from the mean we can find in this group.
# So we only consider the absolute value of this result and we determine what is the maximum
# a certain college have in this group.

def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [35]:
# Only the function name is passed to the agg groupby method.
# It will automatically pass as a parameter to this function each Series that constitutes each group aggreagting column (here
# only one column 'UGDS'). Finally, one value will be returned for each group and all the values will be summarised
# in a Series.

college.groupby('STABBR')['UGDS'].agg(max_deviation) \
                                 .round(1).head()

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
Name: UGDS, dtype: float64

In [36]:
# Si l'on veut un DF
pd.DataFrame(college.groupby('STABBR')['UGDS'].agg(max_deviation) \
                                 .round(1)).head()

,UGDS
STABBR,
AK,2.6
AL,5.8
AR,6.3
AS,NaN
AZ,9.9


#### It is possible to apply our customized function to multiple aggregating columns. We simply add more column names to the indexing operator. The max_deviation function only works with numeric columns:

In [37]:
college.groupby('STABBR')['UGDS', 'SATVRMID', 'SATMTMID'] \
       .agg(max_deviation).round(1).head()

,UGDS,SATVRMID,SATMTMID
STABBR,,,
AK,2.6,NaN,NaN
AL,5.8,1.6,1.8
AR,6.3,2.2,2.3
AS,NaN,NaN,NaN
AZ,9.9,1.9,1.4


In [38]:
college.groupby(['STABBR', 'RELAFFIL']) \
        ['UGDS', 'SATVRMID', 'SATMTMID'] \
       .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                max_deviation    mean     std max_deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

In [39]:
max_deviation.__name__ = 'Max Deviation'
college.groupby(['STABBR', 'RELAFFIL']) \
        ['UGDS', 'SATVRMID', 'SATMTMID'] \
       .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                Max Deviation    mean     std Max Deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                Max Deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

## Customizing aggregating functions with *args and **kwargs

In [40]:
college = pd.read_csv('master/data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])

In [41]:
import inspect
inspect.signature(grouped.agg)

<Signature (arg, *args, **kwargs)>

In [42]:
coldic = pd.read_csv('master/data/college_data_dictionary.csv')
coldic.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
column_name,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
description,Institution Name,City Location,State Abbreviation,Historically Black College or University,0/1 Men Only,0/1 Women only,0/1 Religious Affiliation,SAT Verbal Median,SAT Math Median,Distance Education Only,Undergraduate Enrollment,Percent Undergrad White,Percent Undergrad Black,Percent Undergrad Hispanic,Percent Undergrad Asian,Percent Undergrad American Indian/Alaskan Native,Percent Undergrad Native Hawaiian/Pacific Isla...,Percent Undergrad 2 or more races,Percent Undergrad non-resident aliens,Percent Undergrad race unknown,Percent Students part-time,0/1 Currently Operating,Percent Students with Pell grant,Percent Students with federal loan,Percent Students Older than 25,Median Earnings 10 years after enrollment,Median debt of completers


#### In this recipe, we build a customized function for the college dataset that finds the percentage of schools by state and religious affiliation that have an undergraduate population between two values.

In [43]:
def pct_between_1_3k(s):
    return s.between(1000, 3000).mean()

In [44]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
       .agg(pct_between_1_3k).head(9)

STABBR  RELAFFIL
AK      0           0.142857
        1           0.000000
AL      0           0.236111
        1           0.333333
AR      0           0.279412
        1           0.111111
AS      0           1.000000
AZ      0           0.096774
        1           0.000000
Name: UGDS, dtype: float64

#### Let's create a new function that allows the user to define the lower and upper bound

In [45]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

In [46]:
# Faisons un petit test d'utilisation de notre fonction avec une Series quelconque:
ind_exp2

AIRLINE
AA     5.542661
AS    -0.833333
B6     8.692593
DL     0.339691
EV     7.034580
F9    13.630651
HA     4.972973
MQ     6.860591
NK    18.436070
OO     7.593463
UA     7.765755
US     1.681105
VX     5.348884
WN     6.397353
Name: ARR_DELAY, dtype: float64

In [47]:
pct_between(ind_exp2, 2, 8)

0.5714285714285714

In [48]:
# Here the magic of *args and **kwargs come into play.
# In this example, we pass two non-keyword arguments, 1,000 and 10,000, to the agg method.
# Pandas passes these two arguments respectively to the low and high parameters of pct_between.

college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
       .agg(pct_between, 1000, 10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

In [49]:
# We could also have explicitly used the parameter names with the following command to produce the same result:

college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
       .agg(pct_between, high= 10000, low= 1000).head()

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
Name: UGDS, dtype: float64

In [50]:
# Further still, we can mix non-keyword and keyword arguments as long as the keyword arguments come last:

college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
       .agg(pct_between, 1000, high=10000).head()

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
Name: UGDS, dtype: float64

#### Impossible to use multiple aggregation

In [51]:
# college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
#        .agg(['mean', pct_between], low=100, high=1000)

# TypeError: pct_between() missing 2 required positional arguments: 'low' and 'high'

#### we can define a special type of nested function called a closure. We can use a generic closure to build all of our customized functions:

In [52]:
# Il est malheuresement impossible d'utiliser ces arguments supplémentaires lorsque l'on passe plusieurs fonctions
# d'aggrégations.
# La solution est de créer des fonction en amont avec les arguments supllémentaires deja
# "empaquetés", "wrapped" dans la fonction, comme Laura Palmer dans son sac en plastique.
# Nous allons donc definir un fonction speciale appellée closure qui aura pour role d'empaqueter une fonction et
# ses arguments de manière a pouvoir passer ce package dans .agg

def make_agg_func(func, name, *args, **kwargs):
    
#     Note: l'expression  ', *args, **kwargs'   peut s'interpreter vaguement ainsi:
#             "Ainsi qu'un certain nombre d'arguments sans keywords ou avec keyword sachant que
#                 - les arguments sans kw devront être placés avant ceux avec kw
#                 - le premier argument sans kw rencontré sera considéré comme le premier des *args,
#                 - le 2éme argument rencontré comme le second des *args, et ainsi de suite...
    
    # La fonction wrapper retournée à la fin sera la copie de la fonction passée en argument de make_agg_func
    # mais avec ses arguments déjà hardcodés en elle.
    # Par exemple ici nous allons par la suite utiliser notre closure make_agg_func pour créer 2 fonctions à partir de
    # de pct_between qui prend comme argument une Series et 2 bornes low et high.
    # Nous desirons obtenir au final une fonction qui, tout comme la fonction pct_between_1_3k utilisé
    # au debut de cette recipe, ne prenne en argument qu'une Series - les bornes étant déjà harcodées dans la fonction.

    # Nous definissons donc notre wrapper comme une fonction à un seul argument - une Series - qui sera la 
    # copie de la fonction func passée en argument de notre closure make_agg_func (dans notre exemple func sera
    # pct_between)
    # Cette fonction func (pct-between) prend, nous l'avons definie ainsi prédemment, 3 arguments: une Series, un low et un high.
    # Lorsque nous écrivons *args, **kwargs) dans func, nous signifions donc: "Ainsi que les autres arguments
    # de cette fonction func", sachant que nous auront definit la valeur de ces 2 arguments dans make_agg_func.
    
    # Enfin nous modifions le nom interne de ce wrapper pour qu'il puisse être utilisé pour le nommage
    # des colonnes par la suite le cas échéant.
    
    def wrapper(x):
        return func(x, *args, **kwargs)
    wrapper.__name__ = name
    return wrapper

In [53]:
my_agg1 = make_agg_func(pct_between, 'pct_1_3k', low=1000, high=3000)
my_agg2 = make_agg_func(pct_between, 'pct_10_30k', 10000, 30000)

In [54]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
       .agg(['mean', my_agg1, my_agg2]).head()

# Nous sommes donc bien parvenus à passer plusieurs fonctions d'aggrégations, dont certaines customisées,
# et avec des arguments additionnels. Il nous a juste suffit de procéder en 2 étapes:
#     - Empaqueter nos fonction customisées et leurs arguments grace à notre closure.
#     - Faire un groupby classique comme nous savons si bien le faire.

mean  pct_1_3k  pct_10_30k
STABBR RELAFFIL                                   
AK     0         3508.857143  0.142857    0.142857
       1          123.333333  0.000000    0.000000
AL     0         3248.774648  0.236111    0.083333
       1          979.722222  0.333333    0.000000
AR     0         1793.691176  0.279412    0.014706

## Examining the groupby object
#### In this recipe, we examine the groupby object itself by directly calling methods on it as well as iterating through each of its groups.

In [55]:
college = pd.read_csv('master/data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

pandas.core.groupby.groupby.DataFrameGroupBy

In [56]:
dir(grouped)

['CITY',
 'CURROPER',
 'DISTANCEONLY',
 'GRAD_DEBT_MDN_SUPP',
 'HBCU',
 'INSTNM',
 'MD_EARN_WNE_P10',
 'MENONLY',
 'PCTFLOAN',
 'PCTPELL',
 'PPTUG_EF',
 'RELAFFIL',
 'SATMTMID',
 'SATVRMID',
 'STABBR',
 'UG25ABV',
 'UGDS',
 'UGDS_2MOR',
 'UGDS_AIAN',
 'UGDS_ASIAN',
 'UGDS_BLACK',
 'UGDS_HISP',
 'UGDS_NHPI',
 'UGDS_NRA',
 'UGDS_UNKN',
 'UGDS_WHITE',
 'WOMENONLY',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_accessors',
 '_add_numeric_operations',
 '_agg_doc',
 '_aggregate',
 '_aggregate_generic',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply

In [57]:
# Syntaxe étrange, on place ici une boucle for entre des crochets dans un print precédée de la sortie, jamais vu avant.
# On obtiens un retour sous forme de liste:

print([attr for attr in dir(grouped) if not attr.startswith('_')])

# Une manière plus conventionnelle de procéder:

# for attr in dir(grouped):
#     if not attr.startswith('_'):
#         print(attr)
        
# Mais cela ne donnes pas exactement le même resultat, le retour est une longue serie de print.

['CITY', 'CURROPER', 'DISTANCEONLY', 'GRAD_DEBT_MDN_SUPP', 'HBCU', 'INSTNM', 'MD_EARN_WNE_P10', 'MENONLY', 'PCTFLOAN', 'PCTPELL', 'PPTUG_EF', 'RELAFFIL', 'SATMTMID', 'SATVRMID', 'STABBR', 'UG25ABV', 'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITE', 'WOMENONLY', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


In [58]:
grouped.ngroups

112

In [59]:
# the groups attribute contains a dictionary of each unique group
# mapped to all the corresponding index labels of that group:

grouped.groups

{('AK', 0): Int64Index([60, 62, 63, 65, 66, 67, 5171], dtype='int64'),
 ('AK', 1): Int64Index([61, 64, 5417], dtype='int64'),
 ('AL',
  0): Int64Index([   0,    1,    3,    4,    5,    6,    7,    8,    9,   11,   13,
               14,   15,   17,   18,   19,   20,   21,   22,   25,   26,   27,
               28,   29,   31,   32,   33,   34,   37,   38,   39,   40,   42,
               44,   45,   48,   49,   50,   52,   55,   56,   57,   58,   59,
             4307, 4415, 4483, 4484, 4851, 4872, 4986, 4987, 5202, 5296, 5523,
             5564, 5879, 5885, 5890, 5924, 5997, 6013, 6062, 6124, 6126, 6134,
             6139, 6600, 6685, 6799, 6894, 7079],
            dtype='int64'),
 ('AL',
  1): Int64Index([   2,   10,   12,   16,   23,   24,   30,   35,   36,   41,   46,
               47,   51,   53,   54, 5818, 6753, 7061, 7164, 7165, 7166, 7167,
             7168, 7495],
            dtype='int64'),
 ('AR',
  0): Int64Index([ 128,  129,  130,  132,  133,  135,  136,  137,  138,  139

In [60]:
grouped.groups.keys()

dict_keys([('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1), ('AS', 0), ('AZ', 0), ('AZ', 1), ('CA', 0), ('CA', 1), ('CO', 0), ('CO', 1), ('CT', 0), ('CT', 1), ('DC', 0), ('DC', 1), ('DE', 0), ('DE', 1), ('FL', 0), ('FL', 1), ('FM', 0), ('GA', 0), ('GA', 1), ('GU', 0), ('GU', 1), ('HI', 0), ('HI', 1), ('IA', 0), ('IA', 1), ('ID', 0), ('ID', 1), ('IL', 0), ('IL', 1), ('IN', 0), ('IN', 1), ('KS', 0), ('KS', 1), ('KY', 0), ('KY', 1), ('LA', 0), ('LA', 1), ('MA', 0), ('MA', 1), ('MD', 0), ('MD', 1), ('ME', 0), ('ME', 1), ('MH', 0), ('MI', 0), ('MI', 1), ('MN', 0), ('MN', 1), ('MO', 0), ('MO', 1), ('MP', 0), ('MS', 0), ('MS', 1), ('MT', 0), ('MT', 1), ('NC', 0), ('NC', 1), ('ND', 0), ('ND', 1), ('NE', 0), ('NE', 1), ('NH', 0), ('NH', 1), ('NJ', 0), ('NJ', 1), ('NM', 0), ('NM', 1), ('NV', 0), ('NV', 1), ('NY', 0), ('NY', 1), ('OH', 0), ('OH', 1), ('OK', 0), ('OK', 1), ('OR', 0), ('OR', 1), ('PA', 0), ('PA', 1), ('PR', 0), ('PR', 1), ('PW', 0), ('RI', 0), ('RI', 1), ('SC', 0),

In [61]:
groups = list(grouped.groups.keys())
groups[:6]

[('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1)]

In [62]:
# Retrieve a single group with the get_group method by passing it a tuple of an exact group label:

grouped.get_group(('FL', 1)).head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Baptist College of Florida,Graceville,FL,0.0,0.0,0.0,1,545.0,465.0,0.0,454.0,0.7996,0.0639,0.0441,0.0044,0.0066,0.0000,0.0308,0.0000,0.0507,0.2291,1,0.5878,0.5602,0.3531,30800,20052
713,Barry University,Miami,FL,0.0,0.0,0.0,1,470.0,462.0,0.0,3913.0,0.2004,0.3177,0.2867,0.0153,0.0038,0.0015,0.0164,0.0741,0.0841,0.1518,1,0.5045,0.6733,0.4361,44100,28250
714,Gooding Institute of Nurse Anesthesia,Panama City,FL,0.0,0.0,0.0,1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,PrivacySuppressed
715,Bethune-Cookman University,Daytona Beach,FL,1.0,0.0,0.0,1,405.0,395.0,0.0,3897.0,0.0190,0.8891,0.0277,0.0021,0.0010,0.0018,0.0198,0.0205,0.0190,0.0523,1,0.7758,0.8867,0.0647,29400,36250
724,Johnson University Florida,Kissimmee,FL,0.0,0.0,0.0,1,480.0,470.0,0.0,220.0,0.7000,0.1136,0.1545,0.0091,0.0000,0.0000,0.0045,0.0045,0.0136,0.1636,1,0.6689,0.7384,0.2185,26300,20199


In [63]:
from IPython.display import display

# We iterate through the groupby object, so at each iteration we are given a tuple containing the group name
# and corresponding DataFrame group without the grouping columns.
# This tuple is unpacked into the variables name and group in a for-loop.

for name, group in grouped:
    print(name)
    display(group.head(3))
    
# If we use the print function also for the DataFrames, we'll be returned plain text without any nice HTML formatting.
# Using the display function instead of print will produce DataFrames in their normal easy-to-read format.

('AK', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,12865.0,0.5747,0.0358,0.0761,0.0778,0.0653,0.0086,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,University of Alaska Fairbanks,Fairbanks,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,5536.0,0.4259,0.0210,0.0522,0.0126,0.1284,0.0027,0.0401,0.0110,0.3060,0.3887,1,0.2263,0.2550,0.4519,36200,19355
63,University of Alaska Southeast,Juneau,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,1428.0,0.4748,0.0119,0.0623,0.0357,0.1029,0.0147,0.0686,0.0049,0.2241,0.5112,1,0.1769,0.1996,0.5550,37400,16875


('AK', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
61,Alaska Bible College,Palmer,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,27.0,0.8519,0.0000,0.0370,0.0000,0.0741,0.0000,0.0370,0.0,0.0000,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacySuppressed
64,Alaska Pacific University,Anchorage,AK,0.0,0.0,0.0,1,555.0,503.0,0.0,275.0,0.5309,0.0291,0.0364,0.0255,0.1855,0.0109,0.0945,0.0,0.0873,0.3745,1,0.3152,0.5297,0.4910,47000,23250
5417,Alaska Christian College,Soldotna,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,68.0,0.0588,0.0000,0.0147,0.0000,0.7794,0.0000,0.0147,0.0,0.1324,0.0735,1,0.8868,0.6792,0.2264,NaN,PrivacySuppressed


('AL', 0)


...

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
2842,Scott College of Cosmetology,Wheeling,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,77.0,0.8961,0.0390,0.0,0.013,0.0,0.0,0.0519,0.0,0.0,0.0000,1,0.5276,0.4961,0.1111,14800,9250
4019,B M Spurr School of Practical Nursing,Glen Dale,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,31.0,1.0000,0.0000,0.0,0.000,0.0,0.0,0.0000,0.0,0.0,0.0000,1,0.4722,0.0000,0.4444,PrivacySuppressed,PrivacySuppressed
4020,Ben Franklin Career Center,Dunbar,WV,0.0,0.0,0.0,0,NaN,NaN,0.0,435.0,0.8989,0.0989,0.0,0.000,0.0,0.0,0.0023,0.0,0.0,0.7885,1,0.1761,0.0000,0.7568,20800,PrivacySuppressed


('WV', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4016,Alderson Broaddus University,Philippi,WV,0.0,0.0,0.0,1,475.0,485.0,0.0,1024.0,0.7354,0.1611,0.0400,0.0127,0.0039,0.0010,0.0098,0.0342,0.0020,0.0342,1,0.4800,0.8945,0.0722,46000,27000
4018,Appalachian Bible College,Mount Hope,WV,0.0,0.0,0.0,1,514.0,487.0,0.0,196.0,0.9643,0.0000,0.0153,0.0000,0.0051,0.0102,0.0000,0.0051,0.0000,0.0612,1,0.4388,0.2770,0.0899,28700,9300
4027,Davis & Elkins College,Elkins,WV,0.0,0.0,0.0,1,485.0,490.0,0.0,839.0,0.6341,0.0453,0.0012,0.0024,0.0048,0.0000,0.0000,0.1466,0.1657,0.0167,1,0.4188,0.6579,0.1133,35000,23840.5


('WY', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
4128,Casper College,Casper,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,2449.0,0.8689,0.0147,0.0523,0.0073,0.0037,0.0020,0.0163,0.0078,0.0269,0.3250,1,0.2188,0.1455,0.3447,34800,10764
4129,Central Wyoming College,Riverton,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,1099.0,0.7097,0.0136,0.0846,0.0055,0.1301,0.0018,0.0300,0.0073,0.0173,0.4377,1,0.1700,0.0967,0.3992,25200,8757
4130,Eastern Wyoming College,Torrington,WY,0.0,0.0,0.0,0,NaN,NaN,0.0,681.0,0.8502,0.0250,0.0881,0.0059,0.0132,0.0073,0.0088,0.0015,0.0000,0.3275,1,0.1729,0.1166,0.2371,25900,10000


In [64]:
# Calling the head method on your groupby object gets the first rows of each group:

grouped.head(2).head(6)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
10,Birmingham Southern College,Birmingham,AL,0.0,0.0,0.0,1,560.0,560.0,0.0,1180.0,0.7983,0.1102,0.0195,0.0517,0.0102,0.0000,0.0051,0.0000,0.0051,0.0017,1,0.1920,0.4809,0.0152,44200,27000
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,84.0,0.7976,0.1310,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,12865.0,0.5747,0.0358,0.0761,0.0778,0.0653,0.0086,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5


#### the following operation selects the first and last rows from each group:

In [65]:
grouped.nth([1, -1]).head(6)

CITY  CURROPER  DISTANCEONLY GRAD_DEBT_MDN_SUPP  HBCU  \
STABBR RELAFFIL                                                                
AK     0          Fairbanks         1           0.0              19355   0.0   
       0             Barrow         1           0.0  PrivacySuppressed   0.0   
       1          Anchorage         1           0.0              23250   0.0   
       1           Soldotna         1           0.0  PrivacySuppressed   0.0   
AL     0         Birmingham         1           0.0            21941.5   0.0   
       0             Dothan         1           0.0  PrivacySuppressed   0.0   

                                                  INSTNM MD_EARN_WNE_P10  \
STABBR RELAFFIL                                                            
AK     0                  University of Alaska Fairbanks           36200   
       0                               Ilisagvik College           24900   
       1                       Alaska Pacific University           47000   
       1                        Alaska Christian College             NaN   
AL     0             University of Alabama at Birmingham           39700   
       0         Alabama College of Osteopathic Medicine             NaN   

                 MENONLY  PCTFLOAN  PCTPELL  PPTUG_EF  SATMTMID  SATVRMID  \
STABBR RELAFFIL                                                             
AK     0             0.0    0.2550   0.2263    0.3887       NaN       NaN   
       0             0.0    0.0000   0.1323    0.6239       NaN       NaN   
       1             0.0    0.5297   0.3152    0.3745     503.0     555.0   
       1             0.0    0.6792   0.8868    0.0735       NaN       NaN   
AL     0             0.0    0.5214   0.3460    0.2607     565.0     570.0   
       0             0.0       NaN      NaN       NaN       NaN       NaN   

                 UG25ABV     UGDS  UGDS_2MOR  UGDS_AIAN  UGDS_ASIAN  \
STABBR RELAFFIL                                                       
AK     0          0.4519   5536.0     0.0401     0.1284      0.0126   
       0          0.6498    109.0     0.0000     0.6881      0.0826   
       1          0.4910    275.0     0.0945     0.1855      0.0255   
       1          0.2264     68.0     0.0147     0.7794      0.0000   
AL     0          0.2422  11383.0     0.0368     0.0022      0.0518   
       0             NaN      NaN        NaN        NaN         NaN   

                 UGDS_BLACK  UGDS_HISP  UGDS_NHPI  UGDS_NRA  UGDS_UNKN  \
STABBR RELAFFIL                                                          
AK     0             0.0210     0.0522     0.0027    0.0110     0.3060   
       0             0.0183     0.0092     0.0459    0.0183     0.0000   
       1             0.0291     0.0364     0.0109    0.0000     0.0873   
       1             0.0000     0.0147     0.0000    0.0000     0.1324   
AL     0             0.2600     0.0283     0.0007    0.0179     0.0100   
       0                NaN        NaN        NaN       NaN        NaN   

                 UGDS_WHITE  WOMENONLY  
STABBR RELAFFIL                         
AK     0             0.4259        0.0  
       0             0.1376        0.0  
       1             0.5309        0.0  
       1             0.0588        0.0  
AL     0             0.5922        0.0  
       0                NaN        0.0

## Filtering for states with a minority majority
#### we use the college dataset to find all the states that have more non-white undergraduate students than white. As this is a dataset from the US, whites form the majority and therefore, we are looking for states with a minority majority.

In [66]:
college = pd.read_csv('master/data/college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
grouped.ngroups

59

In [67]:
college['STABBR'].nunique() # verifying the same number

59

In [68]:
# Par la suite, on passera en fait au paramètre df chacun des groupes constitutif du groupby

def check_minority(df, threshold):
    
    # Nous avons les % de chaque race et le nombre total d'éléves par college et nous voulons
    # determiner les états qui ont le plus d'éléves non-white, nous devons donc:
    
    
    # Cacluler la proportion de non-white dans chaque college
    minority_pct = 1 - df['UGDS_WHITE']
    
    # En déduire le nombre de non-white dans tout l'état
    total_minority = (df['UGDS'] * minority_pct).sum()
    
    # Caluler le total d'élèves dans l'état
    total_ugds = df['UGDS'].sum()
    
    # En déduire la proportion de non-white dans tout l'état
    total_minority_pct = total_minority / total_ugds
    
    # Non renoyer un booléen nous indiquant si oui ou non cette proportion globale de minorité dans l'état
    # est supérieure à un certain seuil entré au préalble en paramètre
    return total_minority_pct > threshold

In [69]:
# ptits test perso sur le df college entier
# Je recherche manuellement ou est le threshold

check_minority(college, 0.48753462)

True

In [70]:
tot_ugds = college['UGDS'].sum().astype(int)
tot_ugds

16200904

In [71]:
tot_min = ((1 - college['UGDS_WHITE']) * college['UGDS']).sum()
tot_min

7898501.594700001

In [72]:
min_pct = tot_min / tot_ugds
min_pct

0.4875346211976814

In [73]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'] \
       .agg(pct_between, 1000, 10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

In [74]:
# ! : The filter method of the groupby object is completely different
# than the DataFrame filter method covered in the Selecting columns with methods recipe from Chap. 2.

# This filter method takes as first parameter a user-defined function (here check_minority)
# and as subsequent parameters, *args and **kwargs qui renvoient aux argument suppélemntaires de la fonction check_minority.
# In other words we can then pass the user-defined function parameters there.

# Mystère: pourquoi est-on contraint de mettre le keyword 'threshold' ici contrairement à 
# .agg(pct_between, 1000, 10000) plus haut? 

college_filtered = grouped.filter(check_minority, threshold=.5)
college_filtered.head(30)

# Nous constatons que l'ordre des lignes n'est pas celui du groupby() mais celui du tableau d'origine
# duquel ont été supprimés les lignes des groupes ne correspondant pas au critère defini (dont la fonction de filtrage
# renvoi False)

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,0.3162,0.4405,0.0763,0.0017,0.0207,0.0046,0.0373,0.0000,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,0.3253,0.0843,0.1566,0.0000,0.0241,0.0000,0.0241,0.0000,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,0.7600,0.0400,0.1200,0.0000,0.0000,0.0400,0.0400,0.0000,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,0.2143,0.0873,0.5794,0.0159,0.0873,0.0079,0.0000,0.0000,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed
American Indian College Inc,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,89.0,0.0449,0.0562,0.1461,0.0225,0.6854,0.0449,0.0000,0.0000,0.0000,0.0787,0,0.7500,0.5375,0.4684,PrivacySuppressed,PrivacySuppressed
American Institute of Technology,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,266.0,0.4737,0.2481,0.1729,0.0226,0.0113,0.0038,0.0414,0.0000,0.0263,0.0000,1,0.4276,0.6477,0.9137,30800,6135
Carrington College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,617.0,0.2674,0.0616,0.5754,0.0113,0.0713,0.0000,0.0049,0.0000,0.0081,0.0000,1,0.7287,0.4754,0.2722,25100,9500
Carrington College-Mesa,Mesa,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,718.0,0.3677,0.0543,0.3886,0.0195,0.1476,0.0056,0.0125,0.0000,0.0042,0.0139,1,0.7123,0.5455,0.4588,25100,9500


In [75]:
# On constate bien que soit tous les colleges d'un état restent dans le DF soit ils sont tous supprimés

college.query("STABBR == 'AZ'").size

3458

In [76]:
college_filtered.query("STABBR == 'AZ'").size

3458

In [77]:
college.query("STABBR == 'AK'").size

260

In [78]:
college_filtered.query("STABBR == 'AK'").size

0

In [79]:
college.shape

(7535, 26)

In [80]:
college_filtered.shape

(3028, 26)

In [81]:
college_filtered['STABBR'].nunique()

20

### TM

In [82]:
college_filtered_20 = grouped.filter(check_minority, threshold=.2)
college_filtered_20.shape

(7461, 26)

In [83]:
college_filtered_20['STABBR'].nunique()

57

In [84]:
college_filtered_70 = grouped.filter(check_minority, threshold=.7)
college_filtered_70.shape

(957, 26)

In [85]:
college_filtered_70['STABBR'].nunique()

10

## Transforming through a weight loss bet
#### In this recipe, we use simulated data from two individuals to track the percentage of weight loss over the course of four months. At the end of each month, the winner is the one who lost the highest percentage of body weight for that month. To track weight loss, we group our data by month and person, then call the transform method to find the percentage weight loss at each week from the start of the month.

In [86]:
weight_loss = pd.read_csv('master/data/weight_loss.csv')
weight_loss

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190
8,Bob,Feb,Week 1,283
9,Amy,Feb,Week 1,190


In [87]:
# 2 personnes * 4 semaines * 4 mois = 32 lignes
len(weight_loss)

32

In [88]:
weight_loss.query('Month  == "Jan"')

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


In [89]:
def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

# Cette fonction est donc désignée telle que si on lui passe un mois de données d'un concurrent, soit un groupe ou une Serie
# de 4 valeurs, elle renvoi également une Serie de 4 valeurs correspondant aux pourcentage de perte de poid pour chauqe semaine
# du mois

In [90]:
# Nous testons notre fonction:

bob_jan = weight_loss.query('Name=="Bob" and Month=="Jan"')
bob_jan

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
2,Bob,Jan,Week 2,288
4,Bob,Jan,Week 3,283
6,Bob,Jan,Week 4,283


In [91]:
# Il y a une imprecisition dans l'énoncé ici: si les poids sont prelevés en fin de semaine,
# nous ne calculons ainsi la perte de poids que sur 3 semaines.

find_perc_loss(bob_jan['Weight'])

0    0.000000
2   -0.010309
4   -0.027491
6   -0.027491
Name: Weight, dtype: float64

In [92]:
# Je crée en amont les variables pour plus de clarté.
# D'abord je crée un groupby object, qu'on ne peux malheuersment que sommairement inspecter car nous ne definissons
# a ce stade aucune fonction d'aggregation.

weight_loss_gr = weight_loss.groupby(['Name', 'Month'])['Weight']

In [93]:
# 2 personnes et 4 mois, nous aurons donc 8 groupes

weight_loss_gr.ngroups

8

In [94]:
list(weight_loss_gr.groups)

[('Amy', 'Apr'),
 ('Amy', 'Feb'),
 ('Amy', 'Jan'),
 ('Amy', 'Mar'),
 ('Bob', 'Apr'),
 ('Bob', 'Feb'),
 ('Bob', 'Jan'),
 ('Bob', 'Mar')]

In [95]:
weight_loss

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190
8,Bob,Feb,Week 1,283
9,Amy,Feb,Week 1,190


In [96]:
weight_loss_gr.agg('mean')

Name  Month
Amy   Apr      164.75
      Feb      181.00
      Jan      191.25
      Mar      171.50
Bob   Apr      255.50
      Feb      273.50
      Jan      286.25
      Mar      266.25
Name: Weight, dtype: float64

In [97]:
# Pourquoi ici est-il impossible d'utiliser .agg() et doit-on utiliser .transform()?
# 
# Rappellons ce qu'il se passe lorsque nous operons un groupby() avec .agg():
# 
#     - D'abord .groupby() regroupe toutes les lignes du tableau selon les valeurs uniques des grouping columns
#       spécifiées (pas de changement de len), et isole les aggregating columns (changement du nombre de colonnes)
# 
#     - Puis la fonction d'aggrégation effectue une certaine opération sur les groupes des aggregating columns et renvoi une
#       valeur unique pour chaque groupe.
# 
#     - Etant donné que nous avons correctement indiqué la manière dont nous souhaitons aggréger les valeurs des AGG-COL
#       de chaque groupe, il est à présent possible d'aggreger chaque groupe en une seule ligne. Le DF final aura donc
#       autant de lignes que le nombre de groupes constitutifs du groupby()
#
#    - Notons que la repartition par groupes du groupby() aura servit 2 fois: une fois pour indiquer sur quelles valeurs
#      effectuer les calculs puis une autre fois pour designer les colonnes à aggreger en une seule.
# 
# 
# Rappellons également ce qu'il se passe pour filter():
# 
#     - regroupage idem
# 
#     - operations idem sauf qu'on obtiens pour chaque groupe un booléen et non une valeur
# 
#     - le retour est le DF d'origine duquel on été supprimés les lignes de groupes qui ont renvoyé False.
#       Le DF final aura donc moins (ou autant) de lignes que le DF d'origine
# 
# 
# Etudions ce qu'il se passe pour transform():
# 
#     - regroupage idem
# 
#     - operations idem sauf que ce n'est plus une unique valeur qui est retournée pour chaque groupe mais un nombre de valeurs
#       identique à celui du groupe passé. 
# 
#     -  Le DF final a donc autant de lignes que le DF d'origine et les lignes conservent l'ordre du DF d'origine
# 
#     - Le groupage n'aura donc ici servit qu'une fois pour designer les groupes à partir
#       desquels on calcule chaque valeur.
# 

pcnt_loss = weight_loss.groupby(['Name', 'Month'])['Weight'].transform(find_perc_loss)
pcnt_loss

0     0.000000
1     0.000000
2    -0.010309
3    -0.040609
4    -0.027491
5    -0.040609
6    -0.027491
7    -0.035533
8     0.000000
9     0.000000
10   -0.028269
11   -0.031579
12   -0.053004
13   -0.068421
14   -0.053004
15   -0.089474
16    0.000000
17    0.000000
18    0.011194
19    0.000000
20   -0.011194
21   -0.017341
22   -0.026119
23   -0.017341
24    0.000000
25    0.000000
26   -0.011494
27   -0.035294
28   -0.030651
29   -0.035294
30   -0.042146
31   -0.052941
Name: Weight, dtype: float64

In [98]:
weight_loss['Perc Weight Loss'] = pcnt_loss.round(3)

# Pour raccourcir le retour nous ne selectionnons que les 2 premiers mois de Bob:
weight_loss.query('Name=="Bob" and Month in ["Jan", "Feb"]')

,Name,Month,Week,Weight,Perc Weight Loss
0,Bob,Jan,Week 1,291,0.000
2,Bob,Jan,Week 2,288,-0.010
4,Bob,Jan,Week 3,283,-0.027
6,Bob,Jan,Week 4,283,-0.027
8,Bob,Feb,Week 1,283,0.000
10,Bob,Feb,Week 2,275,-0.028
12,Bob,Feb,Week 3,268,-0.053
14,Bob,Feb,Week 4,268,-0.053


In [99]:
# Pour determiner le gagnant de chaque mois seule la dernière semaine de chaque mois nous interresse:
week4 = weight_loss.query('Week == "Week 4"')
week4

,Name,Month,Week,Weight,Perc Weight Loss
6,Bob,Jan,Week 4,283,-0.027
7,Amy,Jan,Week 4,190,-0.036
14,Bob,Feb,Week 4,268,-0.053
15,Amy,Feb,Week 4,173,-0.089
22,Bob,Mar,Week 4,261,-0.026
23,Amy,Mar,Week 4,170,-0.017
30,Bob,Apr,Week 4,250,-0.042
31,Amy,Apr,Week 4,161,-0.053


In [100]:
winner = week4.pivot(index='Month', columns='Name',
                     values='Perc Weight Loss')
winner

Name,Amy,Bob
Month,,
Apr,-0.053,-0.042
Feb,-0.089,-0.053
Jan,-0.036,-0.027
Mar,-0.017,-0.026


In [101]:
winner['Winner'] = np.where(winner['Amy'] < winner['Bob'],
                            'Amy', 'Bob')
winner.style.highlight_min(axis=1)

Name,Amy,Bob,Winner
Month,,,
Apr,-0.053,-0.042,Amy
Feb,-0.089,-0.053,Amy
Jan,-0.036,-0.027,Amy
Mar,-0.017,-0.026,Bob


In [102]:
winner.Winner.value_counts()

Amy    3
Bob    1
Name: Winner, dtype: int64

### TM

In [103]:
week4

,Name,Month,Week,Weight,Perc Weight Loss
6,Bob,Jan,Week 4,283,-0.027
7,Amy,Jan,Week 4,190,-0.036
14,Bob,Feb,Week 4,268,-0.053
15,Amy,Feb,Week 4,173,-0.089
22,Bob,Mar,Week 4,261,-0.026
23,Amy,Mar,Week 4,170,-0.017
30,Bob,Apr,Week 4,250,-0.042
31,Amy,Apr,Week 4,161,-0.053


In [104]:
week4a = week4.copy()

# We want a list or an array of the months in chronological order.
# As the Month column is already in chronological order, we can simply use the unique method,
# which preserves order to get the array that we desire.

month_chron = week4a['Month'].unique() # or use drop_duplicates
month_chron

array(['Jan', 'Feb', 'Mar', 'Apr'], dtype=object)

In [105]:
# To convert the Month column, we use the Categorical constructor.
# Then we pass it the original column as a Series and a unique sequence of all the categories in the
# desired order to the categories parameter.

week4a['Month'] = pd.Categorical(week4a['Month'],
                                 categories=month_chron,
                                 ordered=True)
week4a

,Name,Month,Week,Weight,Perc Weight Loss
6,Bob,Jan,Week 4,283,-0.027
7,Amy,Jan,Week 4,190,-0.036
14,Bob,Feb,Week 4,268,-0.053
15,Amy,Feb,Week 4,173,-0.089
22,Bob,Mar,Week 4,261,-0.026
23,Amy,Mar,Week 4,170,-0.017
30,Bob,Apr,Week 4,250,-0.042
31,Amy,Apr,Week 4,161,-0.053


In [106]:
# In general, to sort columns of object data type
# by something other than alphabetical, convert them to categorical.

week4a.pivot(index='Month', columns='Name',
             values='Perc Weight Loss')

Name,Amy,Bob
Month,,
Jan,-0.036,-0.027
Feb,-0.089,-0.053
Mar,-0.017,-0.026
Apr,-0.053,-0.042


## Calculating weighted mean SAT scores per state with apply
#### In this recipe, we calculate the weighted average of both the math and verbal SAT scores per state from the college dataset. We weight the scores by the population of undergraduate students per school.

In [107]:
college = pd.read_csv('master/data/college.csv')
subset = ['UGDS', 'SATMTMID', 'SATVRMID'] # just a list of columns names

In [108]:
# We must use the dropna() subset parameter to limit the columns it looks at for missing values
# by passing a list of columns names:

college2 = college.dropna(subset=subset)

In [109]:
college.shape

(7535, 27)

In [110]:
college2.shape

(1184, 27)

In [111]:
print('Total dropped: ' + str(len(college) - len(college2)))
print('UGDS: ' + str(college['UGDS'].isna().sum()))
print('SATMTMID: ' + str(college['SATMTMID'].isna().sum()))
print('SATVRMID: ' + str(college['SATVRMID'].isna().sum()))

Total dropped: 6351
UGDS: 661
SATMTMID: 6339
SATVRMID: 6350


In [112]:
college2[['UGDS', 'SATMTMID', 'SATVRMID']].isna().sum()

UGDS        0
SATMTMID    0
SATVRMID    0
dtype: int64

In [113]:
# On appelle notre parametre de fonction df mais c'est en fait des groupes de groupby object
# qui lui seront ici passés

def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum()) # Une seule valeur est retournée, la moyenne pondérée du groupe

In [114]:
# la fonction weighted_math_average est appliquée a chaque groupe et la valeur correpondante est mappée
# à l'index du groupe (cad a la grouping column qui est devenue index) dans une Series

# Nous constatons qu'ici nous n'avons pas d'aggregating column car les colonnes sur lesquels sont basés les
# calculs sont directement utilisées dans weighted_math_average

college2.groupby('STABBR').apply(weighted_math_average).head()

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
dtype: int64

In [115]:
college2.groupby('STABBR').agg(weighted_math_average).head()

# The weighted_math_average function gets applied to each non-grouping column in the DataFrame,
# which is not really the desired result

,INSTNM,CITY,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
STABBR,,,,,,,,,,,,,,,,,,,,,,,,,,
AK,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503
AL,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536,536
AR,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529
AZ,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569,569
CA,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564,564


In [116]:
# If we try to limit the columns to just SATMTMID, then the groupby method kind of
# drop the other columns before calculations. Hence we will get an error as we won't have access anymore to UGDS.
# So, the best way to complete operations that act on multiple columns is with apply, like we did before.

# college2.groupby('STABBR')['SATMTMID'] \
#         .agg(weighted_math_average).head()
# KeyError: 'UGDS'

In [117]:
from collections import OrderedDict
def weighted_average(df):
    
    data = OrderedDict()
    
    # A normal Python dictionary cannot be used to store the data since it does not preserve insertion order.
    
    weight_m = df['UGDS'] * df['SATMTMID']  # a Series
    weight_v = df['UGDS'] * df['SATVRMID']  # a Series
    
    wm_avg = weight_m.sum() / df['UGDS'].sum()  # a single scalar value
    wv_avg = weight_v.sum() / df['UGDS'].sum()  # a single scalar value
    
    data['weighted_math_avg'] = wm_avg
    data['weighted_verbal_avg'] = wv_avg
    data['math_avg'] = df['SATMTMID'].mean()
    data['verbal_avg'] = df['SATVRMID'].mean()
    data['count'] = len(df)
    
    # Ici ce n'est plus une seule valeur qui est finalement retournée (comme précédemment) mais une Series avec index
    
    return pd.Series(data, dtype='int')

In [118]:
# Recapitulons:
    
#     - Nous pouvons passer aux aggregating functions(agg, filter, transform, and apply) des fonction customisées
#     - Nous ne passons alors que le nom de la fonction sans ().
#     - Le parametre "manquant" sera en fait chaque groupe qui constitue le groupby object auquel
#       la fonction d'aggregation est appliquée

college2.groupby('STABBR').apply(weighted_average).head(10)

# Pour chaque groupe est donc indiqué toutes les données stockées dans le OrderedDict et les Keys sont devenues
# les colonnes d'un DF

,weighted_math_avg,weighted_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503,555,503,555,1
AL,536,533,504,508,21
AR,529,504,515,491,16
AZ,569,557,536,538,6
CA,564,539,562,549,72
CO,553,547,540,537,14
CT,545,533,522,517,14
DC,621,623,588,589,6
DE,569,553,495,486,3


In [119]:
from scipy.stats import gmean, hmean
def calculate_means(df):
    df_means = pd.DataFrame(index=['Arithmetic', 'Weighted',
                                   'Geometric', 'Harmonic'])
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        arithmetic = df[col].mean()
        weighted = np.average(df[col], weights=df['UGDS'])
        geometric = gmean(df[col])
        harmonic = hmean(df[col])
        df_means[col] = [arithmetic, weighted,
                         geometric, harmonic]
        df_means['count'] = len(df)
        return df_means.astype(int)

In [120]:
college2.groupby('STABBR').apply(calculate_means).head(12)

SATMTMID  count
STABBR                            
AK     Arithmetic       503      1
       Weighted         503      1
       Geometric        503      1
       Harmonic         503      1
AL     Arithmetic       504     21
       Weighted         536     21
       Geometric        500     21
       Harmonic         497     21
AR     Arithmetic       515     16
       Weighted         529     16
       Geometric        514     16
       Harmonic         513     16

## Grouping by continuous variables
#### In this recipe, we explore the flights dataset to discover the distribution of airlines for different travel distances. This allows us, for example, to find the airline that makes the most flights between 500 and 1,000 miles. To accomplish this, we use the pandas cut function to discretize the distance of each flight flown.

In [121]:
flights = pd.read_csv('master/data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [122]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins)
cuts.head()

0     (500.0, 1000.0]
1    (1000.0, 2000.0]
2     (500.0, 1000.0]
3    (1000.0, 2000.0]
4    (1000.0, 2000.0]
Name: DIST, dtype: category
Categories (5, interval[float64]): [(-inf, 200.0] < (200.0, 500.0] < (500.0, 1000.0] < (1000.0, 2000.0] < (2000.0, inf]]

In [123]:
# La fonction pandas cut retourne une Series dont les valeurs sont des objets CategoricalDtype

cuts.dtypes

CategoricalDtype(categories=[(-inf, 200.0], (200.0, 500.0], (500.0, 1000.0], (1000.0, 2000.0], (2000.0, inf]]
              ordered=True)

In [124]:
cuts.value_counts()

(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: DIST, dtype: int64

In [125]:
# Fin du chapitre UNCLEAR à revoir


# The cuts Series can now be used to form groups.
# We then call the value_counts method on the AIRLINE column to find the distribution for each distance group.

flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True) \
                                .round(3)

# Very interestingly, pandas allows you to pass the groupby method any object
# as long as the passed Series (if a Series is passed) have the same lenght as the DF.
# This means that you are able to form groups from something completely unrelated to the current DataFrame.

DIST              AIRLINE
(-inf, 200.0]     OO         0.326
                  EV         0.289
                  MQ         0.211
                  DL         0.086
                  AA         0.052
                  UA         0.027
                  WN         0.009
(200.0, 500.0]    WN         0.194
                  DL         0.189
                  OO         0.159
                  EV         0.156
                  MQ         0.100
                  AA         0.071
                  UA         0.062
                  VX         0.028
                  US         0.016
                  NK         0.012
                  B6         0.007
                  F9         0.005
                  AS         0.001
(500.0, 1000.0]   DL         0.206
                  AA         0.144
                  WN         0.138
                  UA         0.131
                  OO         0.106
                  EV         0.101
                  MQ         0.051
                  F9         

In [126]:
flights.groupby(cuts)['AIR_TIME'].quantile(q=[.25, .5, .75]) \
                                 .div(60).round(2)

DIST                  
(-inf, 200.0]     0.25    0.43
                  0.50    0.50
                  0.75    0.57
(200.0, 500.0]    0.25    0.77
                  0.50    0.92
                  0.75    1.05
(500.0, 1000.0]   0.25    1.43
                  0.50    1.65
                  0.75    1.92
(1000.0, 2000.0]  0.25    2.50
                  0.50    2.93
                  0.75    3.40
(2000.0, inf]     0.25    4.30
                  0.50    4.70
                  0.75    5.03
Name: AIR_TIME, dtype: float64

In [127]:
labels = ['Under an Hour', '1 Hour', '1-2 Hour',
          '2-4 Hours', '4+ Hours']
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)
flights.groupby(cuts2)['AIRLINE'].value_counts(normalize=True) \
                                 .round(3) \
                                 .unstack() \
                                 .style.highlight_max(axis=1)

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
DIST,,,,,,,,,,,,,,
Under an Hour,0.052,nan,nan,0.086,0.289,nan,nan,0.211,nan,0.326,0.027,nan,nan,0.009
1 Hour,0.071,0.001,0.007,0.189,0.156,0.005,nan,0.1,0.012,0.159,0.062,0.016,0.028,0.194
1-2 Hour,0.144,0.023,0.003,0.206,0.101,0.038,nan,0.051,0.03,0.106,0.131,0.025,0.004,0.138
2-4 Hours,0.264,0.016,0.003,0.165,0.016,0.031,nan,0.003,0.045,0.046,0.199,0.04,0.012,0.16
4+ Hours,0.212,0.012,0.08,0.171,nan,0.004,0.028,nan,0.019,nan,0.289,0.065,0.074,0.046


## Counting the total number of flights between cities
#### In this recipe, we count the total number of flights between two cities regardless of which one is the origin or destination. To accomplish this, we sort the origin and destination airports alphabetically so that each combination of airports always occurs in the same order. We can then use this new column arrangement to form groups and then to count.

In [128]:
flights = pd.read_csv('master/data/flights.csv')
flights_ct = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()
flights_ct.head()

ORG_AIR  DEST_AIR
ATL      ABE         31
         ABQ         16
         ABY         19
         ACY          6
         AEX         40
dtype: int64

In [129]:
flights_ct.loc[[('ATL', 'IAH'), ('IAH', 'ATL')]]

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

In [130]:
flights_sort = flights[['ORG_AIR', 'DEST_AIR']] \
                      .apply(sorted, axis=1)
flights_sort.head()

0    [LAX, SLC]
1    [DEN, IAD]
2    [DFW, VPS]
3    [DCA, DFW]
4    [LAX, MCI]
dtype: object

In [131]:
# Traffic pour pouvoir continuer
data_sorted_M = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sort = pd.DataFrame(data_sorted_M, columns=['ORG_AIR', 'DEST_AIR'])
# fin du traffic

flights_sort.head()

,ORG_AIR,DEST_AIR
0,LAX,SLC
1,DEN,IAD
2,DFW,VPS
3,DCA,DFW
4,LAX,MCI


In [132]:
rename_dict = {'ORG_AIR':'AIR1', 'DEST_AIR':'AIR2'}
flights_sort = flights_sort.rename(columns=rename_dict)
flights_ct2 = flights_sort.groupby(['AIR1', 'AIR2']).size()
flights_ct2.head()

AIR1  AIR2
ABE   ATL     31
      ORD     24
ABI   DFW     74
ABQ   ATL     16
      DEN     46
dtype: int64

In [133]:
flights_ct2.loc[('ATL', 'IAH')]

269

In [134]:
# flights_ct2.loc[('IAH', 'ATL')]
# IndexingError: Too many indexers

In [135]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

In [136]:
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
fs_orig = flights_sort.rename(columns={'ORG_AIR':'AIR1',
                                       'DEST_AIR':'AIR2'})
flights_sort2.equals(fs_orig)

True

In [137]:
%%timeit
flights_sort = flights[['ORG_AIR', 'DEST_AIR']] \
                      .apply(sorted, axis=1)

3.23 s ± 48.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])

11.2 ms ± 383 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Finding the longest streak of on-time flights
#### In this recipe, we find the longest consecutive streak of on-time flights for each airline at each origin airport. This requires each value in a column to be aware of the value immediately following it. We make clever use of the diff and cumsum methods in order to find streaks before applying this methodology to each of the groups.

In [139]:
s = pd.Series([0, 1, 1, 0, 1, 1, 1, 0])
s

0    0
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

In [140]:
s1 = s.cumsum()
s1

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64

In [141]:
s.mul(s1)

0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

In [142]:
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -2.0
4    3.0
5    1.0
6    1.0
7   -5.0
dtype: float64

In [143]:
s.mul(s1).diff().where(lambda x: x < 0)

0    NaN
1    NaN
2    NaN
3   -2.0
4    NaN
5    NaN
6    NaN
7   -5.0
dtype: float64

In [144]:
s.mul(s1).diff().where(lambda x: x < 0).ffill() \
 .add(s1, fill_value=0)

0    0.0
1    1.0
2    2.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

In [145]:
flights = pd.read_csv('master/data/flights.csv')
flights['ON_TIME'] = flights['ARR_DELAY'].lt(15).astype(int)
flights[['AIRLINE', 'ORG_AIR']].head(10)

,AIRLINE,ORG_AIR
0,WN,LAX
1,UA,DEN
2,MQ,DFW
3,AA,DFW
4,WN,LAX
5,UA,IAH
6,AA,DFW
7,F9,SFO
8,AA,ORD
9,UA,IAH


In [146]:
def max_streak(s):
    s1 = s.cumsum()
    return s.mul(s1).diff().where(lambda x: x < 0) \
            .ffill().add(s1, fill_value=0).max()

In [147]:
flights.sort_values(['MONTH', 'DAY', 'SCHED_DEP']) \
       .groupby(['AIRLINE', 'ORG_AIR'])['ON_TIME'] \
       .agg(['mean', 'size', max_streak]).round(2).head()

mean  size  max_streak
AIRLINE ORG_AIR                        
AA      ATL      0.82   233          15
        DEN      0.74   219          17
        DFW      0.78  4006          64
        IAH      0.80   196          24
        LAS      0.79   374          29

In [148]:
def max_delay_streak(df):
    df = df.reset_index(drop=True)
    s = 1-df['ON_TIME']
    s1 = s.cumsum()
    streak = s.mul(s1).diff().where(lambda x: x < 0) \
              .ffill().add(s1, fill_value=0)
    last_idx = streak.idxmax()
    first_idx = last_idx - streak.max() + 1
    df_return = df.loc[[first_idx, last_idx], ['MONTH', 'DAY']]
    df_return['streak'] = streak.max()
    df_return.index = ['first', 'last']
    df_return.index.name='type'
    return df_return

In [149]:
flights.sort_values(['MONTH', 'DAY', 'SCHED_DEP']) \
       .groupby(['AIRLINE', 'ORG_AIR']) \
       .apply(max_delay_streak) \
       .sort_values('streak', ascending=False).head(10)

C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if __name__ == '__main__':


MONTH   DAY  streak
AIRLINE ORG_AIR type                      
AA      DFW     first    2.0  26.0    38.0
                last     3.0   1.0    38.0
MQ      ORD     last     1.0  12.0    28.0
                first    1.0   6.0    28.0
        DFW     last     2.0  26.0    25.0
                first    2.0  21.0    25.0
NK      ORD     first    6.0   7.0    15.0
                last     6.0  18.0    15.0
DL      ATL     last    12.0  24.0    14.0
                first   12.0  23.0    14.0